In [29]:
import time
import pandas as pd
import re
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def save_page_html(url, save_path):
    chrome_options = Options()
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)

    # Handle cookies if there is a popup
    try:
        # Wait for the cookie popup to appear (adjust the timeout as needed)
        cookie_popup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "cookie-popup"))
        )

        # Click the "X" button to close the cookie popup
        close_button = cookie_popup.find_element(By.XPATH, "//button[contains(@class, 'close-button')]")
        close_button.click()
    except Exception as e:
        print(f"Cookie popup not found or encountered an error: {e}")

    # Wait for some time after closing the cookie popup (adjust as needed)
    time.sleep(5)

    # Get the HTML of the page
    page_html = driver.page_source

    # Save the HTML to a file
    with open(save_path, "w", encoding="utf-8") as file:
        file.write(page_html)

    print(f"HTML saved to {save_path}")

    # Close the browser window
    driver.quit()

# Example usage
website_url = "https://www.**********.it/sport/CALCIO/SERIE%20A"
output_path = "last_odds.html"
save_page_html(website_url, output_path)


Cookie popup not found or encountered an error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AD1C82B2+55298]
	(No symbol) [0x00007FF7AD135E02]
	(No symbol) [0x00007FF7ACFF05AB]
	(No symbol) [0x00007FF7AD03175C]
	(No symbol) [0x00007FF7AD0318DC]
	(No symbol) [0x00007FF7AD06CBC7]
	(No symbol) [0x00007FF7AD0520EF]
	(No symbol) [0x00007FF7AD06AAA4]
	(No symbol) [0x00007FF7AD051E83]
	(No symbol) [0x00007FF7AD02670A]
	(No symbol) [0x00007FF7AD027964]
	GetHandleVerifier [0x00007FF7AD540AAB+3694587]
	GetHandleVerifier [0x00007FF7AD59728E+4048862]
	GetHandleVerifier [0x00007FF7AD58F173+4015811]
	GetHandleVerifier [0x00007FF7AD2647D6+695590]
	(No symbol) [0x00007FF7AD140CE8]
	(No symbol) [0x00007FF7AD13CF34]
	(No symbol) [0x00007FF7AD13D062]
	(No symbol) [0x00007FF7AD12D3A3]
	BaseThreadInitThunk [0x00007FF9F64E257D+29]
	RtlUserThreadStart [0x00007FF9F75AAA58+40]

HTML saved to last_odds.html


In [30]:
def filter_words(input_file, output_file, start_date, end_date):
    with open(input_file, 'r', encoding='utf-8') as infile:
        words = infile.read().split()

    start_index = next((i for i, word in enumerate(words) if word == start_date), None)
    end_index = next((i for i, word in enumerate(words) if word == end_date), None)

    # Verifica se le parole '15/12' e '18/12' sono presenti nel file
    if start_index is None or end_index is None:
        raise ValueError("Le date di inizio o fine non sono presenti nel file.")

    # Filtra le parole tra '15/12' e '18/12' inclusi
    filtered_words = words[start_index:end_index + 20644]

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write(' '.join(filtered_words))

# Esempio di utilizzo
input_file_path = "last_odds.html"
output_file_path = "last_odds.html"
start_date_filter = '15/12'
end_date_filter = '18/12'

filter_words(input_file_path, output_file_path, start_date_filter, end_date_filter)


In [31]:
def find_numbers_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Definisci il pattern regex per trovare le stringhe '\d\d,\d\d' o '\d,\d\d'
    pattern = re.compile(r'\b\d{1,2},\d{1,2}\b')

    # Trova tutte le corrispondenze nel testo
    matches = pattern.findall(content)

    return matches

# Esempio di utilizzo
file_path = "last_odds.html"
result = find_numbers_in_file(file_path)

if len(result) != 70:
    raise ValueError(' WARNING, NOT 10 MATCHES SELECTED ')

with open('last_odds.csv', mode='w', newline='', encoding='utf-8',) as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['1','x','2'])
    reset=0
    temp=[]
    for numero in result:
        temp.append(numero.replace(',', '.'))
        reset=reset+1
        if (reset % 7 ==0):
            csv_writer.writerow(temp[::-1][:3])
            print(temp)
            temp=[]
            reset=0


['1.67', '2.10', '2.35', '1.55', '1.80', '3.30', '5.00']
['2.00', '1.73', '1.92', '1.77', '3.55', '3.30', '2.10']
['1.85', '1.87', '1.50', '2.40', '9.00', '5.50', '1.30']
['1.80', '1.90', '1.90', '1.80', '5.75', '3.90', '1.60']
['1.95', '1.77', '1.70', '2.00', '5.75', '4.25', '1.55']
['1.87', '1.85', '1.70', '2.05', '6.75', '4.50', '1.45']
['2.35', '1.53', '1.65', '2.10', '3.05', '3.50', '2.25']
['1.73', '2.00', '2.40', '1.50', '3.25', '3.00', '2.40']
['1.90', '1.80', '1.92', '1.80', '1.75', '3.70', '4.75']
['1.77', '1.93', '1.55', '2.30', '9.00', '5.50', '1.30']


In [32]:
column_types = {'1': float, 'x': float, '2': float}
last_odds = pd.read_csv(r'last_odds.csv', dtype=column_types)
min_odds = pd.read_csv(r'min_odds.csv', dtype=column_types)

# Creazione del terzo DataFrame contenente il minimo dei valori
df_min = pd.DataFrame()

# Itera attraverso le colonne e le righe per ottenere i minimi
for col in last_odds.columns:
    df_min[col] = [min(a, b) for a, b in zip(min_odds[col], last_odds[col])]

df_min.to_csv('min_odds.csv',index=False)

In [33]:
last_odds,  min_odds

(      1     x     2
 0  5.00  3.30  1.80
 1  2.10  3.30  3.55
 2  1.30  5.50  9.00
 3  1.60  3.90  5.75
 4  1.55  4.25  5.75
 5  1.45  4.50  6.75
 6  2.25  3.50  3.05
 7  2.40  3.00  3.25
 8  4.75  3.70  1.75
 9  1.30  5.50  9.00,
       1     x     2
 0  5.00  3.30  1.80
 1  2.10  3.30  3.55
 2  1.30  5.50  9.00
 3  1.60  3.90  5.75
 4  1.55  4.25  5.75
 5  1.45  4.50  6.75
 6  2.25  3.50  3.05
 7  2.40  3.00  3.25
 8  4.75  3.70  1.75
 9  1.30  5.50  9.00)